In [5]:
# Consolidate all passage scores
scorepsgroot='../../data/scores/bypsg3/'
scorepsgconsolidatedfn='../../data/scores/bypsg3.pkl'

In [6]:
import sys; sys.path.append('../..')
from abslithist import *

In [7]:
def consolidate_psg_scores(C,nmin=50):
    il=[]
    for t in tqdm(C.texts()):
        ifn=os.path.join(scorepsgroot,C.name,t.id+'.csv')
        if not os.path.exists(ifn): continue
        idf=pd.read_csv(ifn)
        idf['id']=t.id
        idf['year']=t.year
        idf['field_period']=to_field_period(t.year)
        il+=[idf]
    odf=pd.concat(il).set_index(['id','year','field_period','i_psg','txt'])
    odf=odf.sort_values('val_mean')
    # filter?
    if nmin: odf=odf.query(f'num_recog>={nmin}')
    
    vcol='val_mean'
    xmean=odf[vcol].mean()
    xstd=odf[vcol].std()
    odf['val_mean_z']=odf['val_mean'].apply(lambda x: (x-xmean)/xstd)        
    return odf

In [8]:
if not os.path.exists(scorepsgconsolidatedfn):
    odf=consolidate_psg_scores(C)
    odf.to_pickle(scorepsgconsolidatedfn)
else:
    odf=pd.read_pickle(scorepsgconsolidatedfn)
dfq=odf.reset_index()
dfq=C.meta.reset_index().merge(dfq,on='id',suffixes=('_meta',''))
dfq=dfq.query('canon_name!="Classic Chinese nov."') # too far outside traditions on graph
dfq=dfq.sort_values('val_mean_z')
dfq

100%|██████████| 1809/1809 [00:26<00:00, 68.14it/s]


,id,author,title,year_meta,canon_genre,canon_name,major_genre,setting_elson,year_trans,author_id,...,num_word,num_word_types,ttr,num_recog,num_recog_types,ttr_recog,val_mean,val_median,val_stdev,val_mean_z
413969,chadwyck.Holcroft.The_Adventures_of_Hugh_Tr.1128,"Holcroft, Thomas, 1745-1809",The Adventures of Hugh Trevor,1794,(Anti-)Jacobin nov.,Holcroft,Novel,,,,...,167,86,0.514970,62,46,0.741935,-1.384575,-1.504237,0.584009,-3.065383
304271,chadwyck.Fielding.Tom_Jones.1029,"Fielding, Henry, 1707-1754",Tom Jones (1749),1749,H. Fielding,Fielding,Novel,,,,...,196,94,0.479592,54,35,0.648148,-1.384041,-1.433622,0.427882,-3.064111
12580,chadwyck.Amory.John_Buncle.0993,"Amory, Thomas, 1691?-1788?",John Buncle (1756),1756,,,,,,,...,129,62,0.480620,54,33,0.611111,-1.366873,-1.463607,0.437405,-3.023241
12581,chadwyck.Amory.John_Buncle.0993,"Amory, Thomas, 1691?-1788?",John Buncle (1756),1756,,,,,,,...,150,99,0.660000,50,45,0.900000,-1.351782,-1.474633,0.550533,-2.987313
718048,Johnson.Rambler.Vol1,Samuel Johnson,The Rambler,1750,,,,,,,...,193,118,0.611399,53,50,0.943396,-1.349135,-1.494209,0.596056,-2.981012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667236,Rabelais.Gargantua and Pantagruel,Rabelais,Gargantua and Pantagruel,1534,Rabelais,Rabelais,Other,,,,...,74,64,0.864865,51,48,0.941176,1.591506,1.801656,0.748263,4.019686
667237,Rabelais.Gargantua and Pantagruel,Rabelais,Gargantua and Pantagruel,1534,Rabelais,Rabelais,Other,,,,...,69,61,0.884058,51,43,0.843137,1.660586,1.946955,0.854439,4.184143
667238,Rabelais.Gargantua and Pantagruel,Rabelais,Gargantua and Pantagruel,1534,Rabelais,Rabelais,Other,,,,...,88,72,0.818182,50,46,0.920000,1.703995,1.990096,0.876728,4.287485
667239,Rabelais.Gargantua and Pantagruel,Rabelais,Gargantua and Pantagruel,1534,Rabelais,Rabelais,Other,,,,...,101,92,0.910891,50,43,0.860000,1.709474,1.988936,0.891878,4.300528


In [9]:
# init_css()
# printm('### Most abstract')
# for n in range(5):showpsg_t(dfq.iloc[n].txt, C.textd[dfq.iloc[n].id],incl_distro=False)
# printm('### Most Concrete')
# for n in range(5):
#     n=-(n+1)
#     showpsg_t(dfq.iloc[n].txt, C.textd[dfq.iloc[n].id], incl_distro=False)

### Make sample?

In [10]:
def classpsg(x):
    if x<=-1.5: return 'Abstract'
    if x>=1.5: return 'Concrete'
    return 'Neither'
dfq['decision']=dfq.val_mean_z.apply(classpsg)
dfq['period']=dfq.apply(lambda row: row.field_period if row.year>=1600 else 'Pre-C17', axis=1)
dfq.decision.value_counts()

Neither     820190
Abstract     66345
Concrete     65216
Name: decision, dtype: int64

In [11]:
gby=['period','decision']
gby1=gby+['author']
samplen=100
samplen1=100

dfq2=dfq.query('decision!="Neither"')
dfgrouped=dfq2.groupby(gby1)
# dfq2=dfgrouped.filter(lambda x: len(x)>samplen)
dfsmpl = dfgrouped.sample(n=samplen1,replace=True).drop_duplicates()
print(dfsmpl.shape)
display(dfsmpl.groupby(gby).size())
dfsmpl2=dfsmpl.groupby(gby).sample(n=samplen).drop_duplicates()
display(dfsmpl2.groupby(gby).size())
dfsmpl2=dfsmpl2.sample(frac=1)
dfsmpl2['i_smpl'] = dfsmpl2.groupby(gby).cumcount()
dfsmpl2['id_smpl'] = dfsmpl2.apply(lambda row: f'{row.period}_{row.decision}_{str(row.i_smpl).zfill(2)}', axis=1)
dfsmpl2

(43706, 50)


period   decision
C17      Abstract     1706
         Concrete      829
C18      Abstract     3912
         Concrete      530
C19      Abstract     9542
         Concrete     9812
C20      Abstract     3047
         Concrete    12703
Pre-C17  Abstract      675
         Concrete      950
dtype: int64

period   decision
C17      Abstract    100
         Concrete    100
C18      Abstract    100
         Concrete    100
C19      Abstract    100
         Concrete    100
C20      Abstract    100
         Concrete    100
Pre-C17  Abstract    100
         Concrete    100
dtype: int64

,id,author,title,year_meta,canon_genre,canon_name,major_genre,setting_elson,year_trans,author_id,...,num_recog_types,ttr_recog,val_mean,val_median,val_stdev,val_mean_z,decision,period,i_smpl,id_smpl
60328,chadwyck.Bernard.The_Isle_of_Man.0924,"Bernard, Richard, 1568-1641",The Isle of Man (1627),1627,,Allegory,Allegory,,,,...,43,0.843137,-0.755159,-1.068895,0.746975,-1.566950,Abstract,C17,0,C17_Abstract_00
656925,Plato.Republic,Plato,Republic (trans. Jowett),-375,Socratic dialogues,,Dialogue,,,,...,51,0.944444,-0.737271,-0.825669,0.644833,-1.524366,Abstract,Pre-C17,0,Pre-C17_Abstract_00
806349,markmark.Stein.The_Making_of_Americans.1630,"Stein, Gertrude",The Making of Americans,1925,Stein,,Novel,,,,...,28,0.538462,-0.817459,-0.946475,0.530420,-1.715266,Abstract,C20,0,C20_Abstract_00
848751,chadwyck.Thomas.Sketches_of_Character_and.0799,"Thomas, Frederick W. (Frederick William), 1806...","Sketches of Character, and Tales Founded on Fa...",1849,,,,,,,...,50,0.961538,-0.972266,-1.127033,0.554505,-2.083811,Abstract,C19,0,C19_Abstract_00
38074,markmark.Ballard.Crash.1354,"Ballard, James Graham",Crash,1973,,,,,,,...,55,0.964912,0.663804,0.750859,0.766995,1.811134,Concrete,C20,0,C20_Concrete_00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641126,chadwyck.Orrery.Parthenissa_Part.0932,"Orrery, Roger Boyle, Earl of, 1621-1679","Parthenissa, Part 6 (1669)",1669,“Parthenissa”,“Parthenissa”,Romance,,,,...,51,0.962264,-0.968133,-1.195967,0.638849,-2.073971,Abstract,C17,97,C17_Abstract_97
635985,chadwyck.Newcastle.Assaulted_and_pursued_Cha.0941,"Newcastle, Margaret Cavendish, Duchess of, 162...",Assaulted and pursued Chastity (1656),1656,Cavendish,Cavendish,Romance,,,,...,78,0.951220,-0.744007,-0.904806,0.653760,-1.540401,Abstract,C17,98,C17_Abstract_98
344133,Gomberville.Polexander,Gomberville,Polexandre,1632,Heroic rom.,Idealist novel?,Romance,,,,...,48,0.857143,0.698837,1.025429,1.144998,1.894535,Concrete,C17,99,C17_Concrete_99
51627,chadwyck.Belon.The_Court_Secret.0922,"Belon, P. (Peter)",The Court Secret (1689),1689,,,,,,,...,51,0.910714,-0.975254,-1.235721,0.726481,-2.090923,Abstract,C17,99,C17_Abstract_99


In [12]:
ofn=f'''../../data/scores/bypsg3.smpl.t{int(time.time())}.pkl'''
ofn

'../../data/scores/bypsg3.smpl.t1618650787.pkl'

In [13]:
dfsmpl2.to_pickle(ofn)
print('>> saved:',ofn)

>> saved: ../../data/scores/bypsg3.smpl.t1618650787.pkl
